# Clustering & Geodemographics

A common challenge in data analysis is how to group observations in a data set together in a way that allows for generalisation: _this_ group of observations are similar to one another, _that_ group is dissimilar to this group. But what defines similarity and difference? There is no _one_ answer to that question and so there are many different ways to cluster data, each of which has strengths and weaknesses that make them more, or less, appropriate in different contexts.

## Clustering in Python

The most commonly-used _aspatial_ clustering algorighms are all found in [scikit-learn](http://scikit-learn.org/stable/), so that will be the focus of this practical. But just as there are aspatial and spatial statistics, there are also _spatially-aware_ clustering algorithms to be found in [PySAL](http://pysal.readthedocs.io/en/latest/), the Python Spatial Analysis Library.

### Clustering in sklearn

One organisation recently produced a handy scikit-learn cheatsheet that you should [download](https://s3.amazonaws.com/assets.datacamp.com/blog_assets/Scikit_Learn_Cheat_Sheet_Python.pdf). The terminology used in scikit-learn is rather different from anything you will have encountered before (unless you've studied computer science and, possibly, statistics) so it's worth spending a few minutes mapping what you already know on to the sklearn framework:

  | Continuous | Categorical
- | ---------- | -----------
**Supervised** | Regression | Classification
**Unsupervised** | Dimensionality Reduction | Clustering

So clustering is a form of unsupervised (because we don't train the model on what a 'good' result looks like) and categorical (because we get labels out of the model, not predictors) machine learning. Here's a map to sklearn's algorithms and how to navigate them:

<a href="http://scikit-learn.org/stable/tutorial/machine_learning_map/"><img alt="SciKit-Learn Algorithm Map" src="http://scikit-learn.org/stable/_static/ml_map.png"></a>

### Clustering in PySAL

PySAL is similarly complex and _also_ has a map to help you navigate its complexities -- in this case we're particularly interested in the orange 'branch' of PySAL (labelled clustering!):

![PySAL Map](http://darribas.org/gds_scipy16/content/figs/pysal.png)

### Which Approach is Right?

The reason that there is no 'right' approach (as I said above) is that it all depends on what you're trying to accomplish and how you're _reasoning_ about your problem. The image below highlights the extent to which the different clustering approaches in sklearn can produce different results -- and this is only for the _non-geographic_ algorithms!

<a href="http://scikit-learn.org/stable/modules/clustering.html#clustering"><img alt="Clustering algorithm comparison" src="http://scikit-learn.org/stable/_images/sphx_glr_plot_cluster_comparison_0011.png" /></a>

To think about this in a little more detail:

* If I run an online company and I want to classify my customers on the basis of their product purchases, then I probably don't care much about where they are, only about what they buy. So my clustering approach doesn't need to take geography into account. I might well _discover_ that many of my most valuable customers live in a few areas, but that is a finding, not a factor, in my research.
* Conversely, if I am looking for cancer clusters then I might well care a _lot_ about geography because I want to make sure that I don't overlook a bigger cluster because it's 'hidden' inside an area with lots of non-sufferers. In that case, I want my clusters to take geography into account because I'm looking for agglomerations. That approach might classify an area with a smaller proportion of cancer patients as part of a 'cancer cluster' but that's because it is still significant _because_ of the geography.

So you can undertake a spatial analysis using _either_ approach, it just depends on the role that you think geography should play in producing the clusters in the first place. We'll see this in action today!

## Obtaining Geodemographic Data

For the sake of simplicity -- and to help you build on what you learned last year -- we're going to work with the NS-SeC data for London to try to produce a classification of LSOAs based on the mix of socio-economic groups in each area. You will also find [Geocomputation: A Practical Primer](https://uk.sagepub.com/en-gb/eur/geocomputation/book241023) to be rather useful here: although the implementation is in the R programming language, the concerns and the approach are exactly the same.

### InFuse

In case you've forgotten, nearly the _entire_ Census is available to download from [InFuse](http://infuse2011.ukdataservice.ac.uk/). You will need to download the "NS-SeC (National Statistics Socio-economic Classification)" data set cut by **Age** so that it only includes people between 16 and 74.

#### Demographic Data

The criteria we want are:
* The 2011 Census.
* NS-SeC NS-SeC (National Statistics Socio-economic Classification).
* Select the first data set listed (should be just Age and NS-SeC).
* Tick _only_ the 16-74 age group option.
* Tick the following NS-SeC classifications: Total, 1 (+ 1.1, 1.2); 2 (+ L4, L5, L6); 3; 4 (+ L8, L9); 5 (+ L10, L11); 6; 7; 8; NC (+ L15, L16). 
* Click 'Add' to add these to your selection

You should have 21 category combinations when this is done.

**_Note:_** the 'L' levels are hidden by the '+' buttons -- so to get at 1.1 and 1.2 you need to click on the '+' next to Group 1, and so on.

#### Geographic Areas

We want London LSOAs, which means that you need to:
* Expand the Regions section (click on the '+')
* Expand hte London section (click on the next '+')
* Tick LSOAs (4835 areas)
* Click 'Add' to add these to your selection

### ONS Boundary Data

Now we need to download the LSOA boundary data. A quick Google search on "2011 LSOA boundaries" will lead you to the [Data.gov.uk portal](https://data.gov.uk/dataset/lower_layer_super_output_area_lsoa_boundaries). The rest is fairly straightforward:
* We want 'generalised' because that means that they've removed some of the detail from the boundaries so the file will load (and render) more quickly.
* We want 'clipped' because that means that the boundaries have been clipped to the edges of the land (e.g. the Thames; the 'Full' data set splits the Thames down the middle between adjacent LSOAs).

### Setup

You should drag both the LSOA boundary file and the NS-SeC zipfile into a 'data' directory that is the same directory as this notebook so that they're easy to access. You should then:
* Unzip _only_ the LSOA zipfile.
* Rename the directory containing LSOA data to 'lsoa'.
* Rename the Census zipfile to 'AGE_NSSEC_UNIT.zip'

And we're ready to go!

### Other Sources of Data

If you're more interested in US Census data then there's a nice-looking (I haven't used it) [wrapper to the Census API](https://pypi.python.org/pypi/census).

## Getting to Work

To get started we're going to work with pandas and geopandas -- again, nothing new so far but you'll see we've got some new libraries here.

### Specifying the Kernel

**_Note:_** Before you go any further, we need to check that you've got the right 'Kernel' (virutal environment) specified in Jupyter. At top right it should say "Python [spats]" and that is the environment that we want to work in: spats is short Spatial Analysis and that contains all of the libraries that we need for our research. There are other kernels configured and these can be accessed by clicking on the 'Kernel' menu item and then 'Change Kernel'. This feature is well beyond the scope of this practical, but it basically allows you to run multiple 'versions' of Python with different libraries or versions of libraries installed at the same time.

### Importing the Libraries

In [2]:
import pysal as ps
import pandas as pd
import geopandas as gpd
import seaborn as sns
import sklearn
import os
import re
import numpy as np
import zipfile

import random
random.seed(123456789) # For reproducibility

# Make numeric display a bit easier
pd.set_option('display.float_format', lambda x: '{:,.0f}'.format(x))

# Make sure output is into notebook
%matplotlib inline

### Loading the Data

You may need to make a few adjustments to the path to get the data loaded on your own computer. But notice what we're now able to do here: using the `zipfile` library we can extract a data file (or any other file) from the Zip archive without even having to open it. Saves even more time _and_ disk space!

In [3]:
z = zipfile.ZipFile(os.path.join('data','AGE_NSSEC_UNIT.zip'))

df = pd.read_csv(z.open('Data_AGE_NSSEC_UNIT.csv'), skiprows=[0])
print("Shape of dataframe is {0} by {1} ".format(df.shape[0], df.shape[1]))
df.head(2)

Shape of dataframe is 4835 by 27 


,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Age : Age 16 to 74 - NS-SeC (National Statistics Socio-economic Classification) : Total: NS-SeC (National Statistics Socio-economic Classification) - Unit : Persons,Age : Age 16 to 74 - NS-SeC (National Statistics Socio-economic Classification) : 1. Higher managerial; administrative and professional occupations - Unit : Persons,Age : Age 16 to 74 - NS-SeC (National Statistics Socio-economic Classification) : 1.1 Large employers and higher managerial and administrative occupations - Unit : Persons,Age : Age 16 to 74 - NS-SeC (National Statistics Socio-economic Classification) : 1.2 Higher professional occupations - Unit : Persons,Age : Age 16 to 74 - NS-SeC (National Statistics Socio-economic Classification) : 2. Lower managerial; administrative and professional occupations - Unit : Persons,...,Age : Age 16 to 74 - NS-SeC (National Statistics Socio-economic Classification) : Not classified\ L15 Full-time students - Unit : Persons,Age : Age 16 to 74 - NS-SeC (National Statistics Socio-economic Classification) : L17 Not classifiable for other reasons - Unit : Persons,Age : Age 16 to 74 - NS-SeC (National Statistics Socio-economic Classification) : L4 Lower professional and higher technical occupations - Unit : Persons,Age : Age 16 to 74 - NS-SeC (National Statistics Socio-economic Classification) : L5 Lower managerial and administrative occupations - Unit : Persons,Age : Age 16 to 74 - NS-SeC (National Statistics Socio-economic Classification) : L6 Higher supervisory occupations - Unit : Persons,Age : Age 16 to 74 - NS-SeC (National Statistics Socio-economic Classification) : L8 Employers in small organisations - Unit : Persons,Age : Age 16 to 74 - NS-SeC (National Statistics Socio-economic Classification) : L9 Own account workers - Unit : Persons,Age : Age 16 to 74 - NS-SeC (National Statistics Socio-economic Classification) : L10 Lower supervisory occupations - Unit : Persons,Age : Age 16 to 74 - NS-SeC (National Statistics Socio-economic Classification) : L11 Lower technical occupations - Unit : Persons,Unnamed: 26
0,19263,E01000001,City of London 001A,Lower Super Output Areas and Data Zones,LSOADZ,1221,530,69,461,429,...,55,0,300,103,26,18,50,8,4,nan
1,19264,E01000002,City of London 001B,Lower Super Output Areas and Data Zones,LSOADZ,1196,518,88,430,392,...,60,0,274,88,30,25,60,11,4,nan


#### Tidying Up

OK, that's not the most 'useful' of column headings: quite a lot of typing and redundancy to say the least! But we can use code to clean this up!

Let's step through it:
* We first test for the existence of a column at the far end of the file -- since `pandas` stores its columns as an index, we need to extract the values from the index (which returns a `numpy` array) and then look through it for something match `Unnamed: 26`.
* Next, we create a new list called `colnames` and we initialise it with the names we want to use for columns `Unnamed: 0` through `Unnamed: 4`.
* Then we're going to iterate over the remaining column names to try to turn them into something more useful. Note that we use `len(colnames)` to `len(df.columns)` in the `range` function so that if we add or remove lists items from the line above, or if we decide to download a different data set with a different number of columns, then this code continues to work!
* When all of that is done, we assign the new column labels on to the dataframe. 

Done!

In [7]:
import re

In [ ]:
# If we find this column, this deletes it
if np.where(df.columns.values=='Unnamed: 26')[0]:
    del df['Unnamed: 26']

# Initialise the new columns names list
colnames = ['CDU','GeoCode','GeoLabel','GeoType','GeoType2']
col_desc = {'GeoCode': 'GeoID', 'GeoLabel': 'GeoName'}

# Now tidy up the remaining columns
for i in range(len(colnames), len(df.columns)):
    #print(df.columns[i])
    
    # Strip out useless verbiage
    label = df.columns[i].replace('Age : Age 16 to 74 - NS-SeC (National Statistics Socio-economic Classification) : ','')
    label = label.replace(' - Unit : Persons','')
    #print(label)
    
    # Check for a pattern at the start of the label
    m = re.search("^(L?\d(\.?\d?)?).?\s", label)
    if m is not None:
        #print("'" + m.group(1) + "'")
        group_id = m.group(1)
        group_id = 'Group' + group_id.replace('.','',1)
        col_desc[group_id] = label
        colnames.append(group_id)
        # Debugging
        # print("Mapped '{0}' to '{1}'").format(label, group_id)
    elif label.startswith("Total"):
        col_desc['Total'] = label
        colnames.append('Total')
    elif label.startswith("Not classified"):
        m = re.search("\s(L\d+)\s", label)
        if m is not None:
            group_id = m.group(1)
            group_id = 'Group' + group_id.replace('.','',1)
            col_desc[group_id] = label
            colnames.append(group_id)
            # Debugging
            #print("Mapped '{0}' to '{1}'").format(label, group_id)
        else: 
            col_desc['NC'] = label
            colnames.append('NC')
    else:
        print("Don't know what to do with: " + label)

# Final check
print(colnames)

In [36]:
df.columns = colnames

# These are pointless but were hard to 
# get at before because of the column names!
del(df['GeoType'])
del(df['GeoType2'])

df.head(2)

,CDU,GeoCode,GeoLabel,Total,Group1,Group11,Group12,Group2,Group3,Group4,...,NC,GroupL15,GroupL17,GroupL4,GroupL5,GroupL6,GroupL8,GroupL9,GroupL10,GroupL11
0,19263,E01000001,City of London 001A,1221,530,69,461,429,73,68,...,55,55,0,300,103,26,18,50,8,4
1,19264,E01000002,City of London 001B,1196,518,88,430,392,83,85,...,60,60,0,274,88,30,25,60,11,4


### Loading the Geodata

Unlike the NS-SeC data this is fairly straightforward using geopandas:

In [6]:
gdf = gpd.read_file(os.path.join('data','lsoa','Lower_Layer_Super_Output_Areas_December_2011_Generalised_Clipped__Boundaries_in_England_and_Wales.shp'))
gdf.head(2)

,geometry,lsoa11cd,lsoa11nm,lsoa11nmw,objectid,st_areasha,st_lengths
0,"POLYGON ((532106.8939689073 182011.4758723566,...",E01000001,City of London 001A,City of London 001A,1,"133,321","2,292"
1,"POLYGON ((532748.6236845022 181787.1247154782,...",E01000002,City of London 001B,City of London 001B,2,"226,191","2,434"


## Normalising the Data

One of the challenges of clustering is that the scale of each dimension matters: if you were to try to cluster, for example, [1] how many metres per year a glacier moved with [2] the number of cubic metres by which it grew, then you would only be clustering on variable [2]. 

Why is that? Because glaciers contain millions of cubic metres of ice and will grow or shrink by thousands of cubic metres each year. In contrast, most glaciers move at most a few metres per year. So the sheer scale difference between these two dimensions means that the values of variable 1 dominate the clustering algorithm because they provide a much better 'spread' in the data than variable 2.

To address this we need to normalise the data in some way so that the scales are relatively consistent. Does this all sound a little bit familiar from last year?

In [ ]:
# And finally, let's split this into two separate
# data frames: one for the top-level groups, and 
# one for the lower-level groups.